In [13]:
#Lets have matplotlib "inline"
%matplotlib inline

#Import packages we need
import numpy as np
from matplotlib import animation, rc
from matplotlib import pyplot as plt

#Set large figure sizes
#Note, this prevents nice figures for articles...
rc('figure', figsize=(16.0, 12.0))
rc('animation', html='html5')

In [14]:
import pycuda.autoinit
import pycuda.driver as cuda
from pycuda.compiler import SourceModule
from pycuda import gpuarray
from pycuda.tools import PageLockedMemoryPool

import time

In [15]:
kernel_src = """
__global__ void mandelbrotKernel(float* output, unsigned int pitch, 
            unsigned int nx, unsigned int ny, 
            unsigned int iterations, 
            float x0, float y0, 
            float dx, float dy) {

    //Get thread id of this thread
    int i = blockIdx.x*blockDim.x + threadIdx.x;
    int j = blockIdx.y*blockDim.y + threadIdx.y;

    //Check for out of bounds
    if (i < nx && j < ny) {
        float x = i*dx + x0;
        float y = j*dy + y0;

        float2 z0 = make_float2(x, y);
        float2 z = z0;
        int k = 0;

        //Loop until iterations or until it diverges
        while (z.x*z.x + z.y*z.y < 25.0 && k < iterations) {
            float tmp = z.x*z.x - z.y*z.y + z0.x;
            z.y = 2 * z.x*z.y + z0.y;
            z.x = tmp;
            ++k;
        }

        //Write out result to GPU memory
        if (k < iterations) {
            float* row = (float*)((char*)output + j*pitch);
            row[i] = fmod((k - log(log(sqrt(z.x*z.x + z.y*z.y)) / log(5.0)) / log(2.0)) / 100, 1.0);
        }
        else {
            float* row = (float*)((char*)output + j*pitch);
            row[i] = 0.0f;
        }
    }
}
"""
mandelbrotModule = SourceModule(kernel_src)
mandelbrotKernel = mandelbrotModule.get_function("mandelbrotKernel")
mandelbrotKernel.prepare("Piiiiffff")

memorypool = PageLockedMemoryPool()

In [16]:
def mandelbrot(nx, ny, iterations,
              x0, y0, 
              dx, dy,
              block_width=8, block_height=8):
    num_zooms = len(x0)
    zooms = list(range(num_zooms))
    
    assert num_zooms == len(x0)
    assert num_zooms == len(y0)
    assert num_zooms == len(dx)
    assert num_zooms == len(dy)
    
    #Create block dimensions and grid dimensions
    block = (block_width, block_height, 1)
    grid = (int((nx + block_width - 1) / block_width), int((nx + block_height - 1) / block_height), 1)
    
    #allocate gpu data
    output_gpu = [None]*num_zooms
    for i in range(num_zooms):
        output_gpu[i] = gpuarray.zeros((ny, nx), dtype=np.float32)
        
    #create stream
    stream = cuda.Stream()
    
    #Create timing events
    start_events = [None]*num_zooms
    end_events = [None]*num_zooms
    for i in zooms:
        start_events[i] = cuda.Event()
        end_events[i] = cuda.Event()
        
    #Run kernel and generate images
    def launch(i):
        start_events[i].record(stream)
        mandelbrotKernel.prepared_async_call(grid, block, stream, 
                                            output_gpu[i].ptr, np.uint32(output_gpu[i].strides[0]),
                                            np.uint32(nx), np.uint32(ny), np.uint32(iterations),
                                            np.float32(x0[i]), np.float32(y0[i]),
                                            np.float32(dx[i]), np.float32(dy[i]))
        end_events[i].record(stream)

    enqueue_compute_start = time.time()
    [launch(i) for i in zooms]
    enqueue_compute_end = time.time()
    
    #Synchronize
    sync_compute_start = time.time()
    gpu_time_compute = 0.0;
    for i in zooms:
        end_events[i].synchronize()
        milliseconds = end_events[i].time_since(start_events[i])
        print("Iteration {:d} took {:f} ms".format(i, milliseconds))
        gpu_time_compute += milliseconds
    sync_compute_end = time.time()
    
    print("Compute")
    print("Enqueue:  {:f} s".format(enqueue_compute_end - enqueue_compute_start))
    print("Sync:     {:f} s".format(sync_compute_end - sync_compute_start))
    print("CPU time: {:f} s".format(enqueue_compute_end + sync_compute_end - enqueue_compute_start - sync_compute_start))
    print("GPU time: {:f} s".format(gpu_time_compute * 1.0e-3))

    #Allocate CPU data
    retval = [None]*num_zooms
    for i in range(num_zooms):
        retval[i] = memorypool.allocate((ny, nx), np.float32)
    
    #Download from GPU to CPU
    def download(i):
        start_events[i].record(stream)
        #pycuda.driver.memcpy_htod_async(retval[i], output_gpu[i], stream)
        output_gpu[i].get_async(stream=stream, ary=retval[i])
        end_events[i].record(stream)
    enqueue_dl_start = time.time()
    [download(i) for i in zooms]
    enqueue_dl_end = time.time()
    
    #synchronize
    sync_dl_start = time.time()
    gpu_time_dl = 0.0
    for i in zooms:
        end_events[i].synchronize()
        milliseconds = end_events[i].time_since(start_events[i])
        print("Iteration {:d} took {:f} ms".format(i, milliseconds))
        gpu_time_dl += milliseconds
    sync_dl_end = time.time()
    
    print("Download")
    print("Enqueue:  {:f} s".format(enqueue_dl_end - enqueue_dl_start))
    print("Sync:     {:f} s".format(sync_dl_end - sync_dl_start))
    print("CPU time: {:f} s".format(enqueue_dl_end + sync_dl_end - enqueue_dl_start - sync_dl_start))
    print("GPU time: {:f} s".format(gpu_time_dl * 1.0e-3))
    
    print("========")
    print("Averages")
    print("Enqueue compute:  {:f} ms".format(1.0e3*(enqueue_compute_end - enqueue_compute_start) / num_zooms))
    print("Enqueue download: {:f} ms".format(1.0e3*(enqueue_dl_end - enqueue_dl_start) / num_zooms))
    print("Kernel:           {:f} ms".format(gpu_time_compute / num_zooms))
    print("Download:         {:f} ms".format(gpu_time_dl / num_zooms))
    print("========")
    
    return retval

In [17]:
n = 1024
nx = 3*n
ny = 2*n
iterations = 5000

x_center = -0.75 + 0.0025
y_center = 0.1
factor = 0.95
num_zooms = 50

x0 = np.empty(num_zooms, dtype=np.float32)
y0 = np.empty(num_zooms, dtype=np.float32)
dx = np.empty(num_zooms, dtype=np.float32)
dy = np.empty(num_zooms, dtype=np.float32)

x0[0] = x_center - 1.5
y0[0] = y_center - 1.0
dx[0] = 3.0 / nx
dy[0] = 2.0 / ny

for i in range(1, num_zooms):
    dx[i] = dx[i-1] * factor
    dy[i] = dy[i-1] * factor
    
    x0[i] = x_center - dx[i]*nx/2
    y0[i] = y_center - dy[i]*ny/2
    
    print("{:f} x {:f}".format(dx[i]*nx, dy[i]*ny))
    
results = mandelbrot(nx, ny, iterations, x0, y0, dx, dy)

2.850000 x 1.900000
2.707500 x 1.805000
2.572125 x 1.714750
2.443519 x 1.629012
2.321343 x 1.547562
2.205276 x 1.470184
2.095012 x 1.396675
1.990261 x 1.326841
1.890748 x 1.260499
1.796211 x 1.197474
1.706400 x 1.137600
1.621080 x 1.080720
1.540026 x 1.026684
1.463025 x 0.975350
1.389874 x 0.926582
1.320380 x 0.880253
1.254361 x 0.836241
1.191643 x 0.794429
1.132061 x 0.754707
1.075458 x 0.716972
1.021685 x 0.681123
0.970601 x 0.647067
0.922071 x 0.614714
0.875967 x 0.583978
0.832169 x 0.554779
0.790560 x 0.527040
0.751032 x 0.500688
0.713481 x 0.475654
0.677807 x 0.451871
0.643916 x 0.429278
0.611721 x 0.407814
0.581134 x 0.387423
0.552078 x 0.368052
0.524474 x 0.349649
0.498250 x 0.332167
0.473338 x 0.315558
0.449671 x 0.299781
0.427187 x 0.284792
0.405828 x 0.270552
0.385537 x 0.257024
0.366260 x 0.244173
0.347947 x 0.231964
0.330549 x 0.220366
0.314022 x 0.209348
0.298321 x 0.198881
0.283405 x 0.188937
0.269235 x 0.179490
0.255773 x 0.170515
0.242984 x 0.161989
Iteration 0 took 175

In [18]:
dpi=300

fig = plt.figure(figsize=(nx/dpi, ny/dpi), dpi=dpi)
ax = plt.axes([0, 0, 1, 1])
im = plt.imshow(results[0], origin='lower', cmap="terrain", vmax=1.0, vmin=0.0)
plt.axis('off')
#gca().xaxis.set_major_locator(NullLocator())
#gca().yaxis.set_major_locator(NullLocator())
plt.tight_layout()

def animate(i):
    im.set_data(results[i])

anim = animation.FuncAnimation(fig, animate, interval=150, frames=range(len(results)))
plt.close()

from matplotlib.animation import FFMpegWriter
from IPython.display import display, HTML
writer = FFMpegWriter(fps=25)
anim.save("mandelbrot.mp4", writer=writer)
display(HTML("""
<div align="middle">
<video width="80%" controls>
<source src="{:s}" type="video/mp4">
</video>
</div>
""".format("mandelbrot.mp4")))

/home/martinls/.local/lib/python3.5/site-packages/matplotlib/figure.py:2362: UserWarning: This figure includes Axes that are not compatible with tight_layout, so results might be incorrect.
  warnings.warn("This figure includes Axes that are not compatible "
